In [1]:
import sqlite3
import logging
import threading
import time
from datetime import datetime

# IBKR API imports
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

class OptionOrderApp(EWrapper, EClient):
    """Simple IBKR options order application"""
    
    def __init__(self):
        EClient.__init__(self, self)
        self.next_order_id = None
        self.order_status = {}
        self.option_data = {}
        self.order_placed_event = threading.Event()
        self.contract_details_received = threading.Event()
        self.market_data_received = threading.Event()
        self.connected = False
    
    def nextValidId(self, orderId):
        """Callback for next valid order ID"""
        self.next_order_id = orderId
        self.connected = True
        logger.info(f"Connected to IBKR, next order ID: {orderId}")
    
    def error(self, reqId, errorCode, errorString, *args):
        """Error handling callback"""
        if errorCode != 2104 and errorCode != 2106:  # Filter out common notifications
            logger.error(f"Error {errorCode}: {errorString}")
    
    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, *args):
        """Order status update callback"""
        self.order_status[orderId] = {
            'status': status,
            'filled': filled,
            'remaining': remaining,
            'avgFillPrice': avgFillPrice,
            'time': datetime.now(),
        }
        logger.info(f"Order {orderId} status: {status}, filled: {filled}, price: {avgFillPrice}")
        
        if status in ["Submitted", "Filled", "Cancelled"]:
            self.order_placed_event.set()
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Price data callback"""
        if reqId not in self.option_data:
            self.option_data[reqId] = {}
        
        # 1 = bid, 2 = ask, 4 = last, 9 = close
        self.option_data[reqId][tickType] = price
        
        # If we have bid and ask prices, signal that we have market data
        if 1 in self.option_data[reqId] and 2 in self.option_data[reqId]:
            self.market_data_received.set()

def create_option_contract(ticker, expiry_date, strike, option_type):
    """Create an option contract object"""
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "OPT"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = expiry_date
    contract.strike = float(strike)
    contract.right = option_type
    contract.multiplier = "100"
    return contract

def connect_to_ibkr(host='127.0.0.1', port=7497, client_id=1, timeout=10):
    """Connect to IBKR TWS/Gateway"""
    app = OptionOrderApp()
    try:
        app.connect(host, port, client_id)
        
        # Start API thread
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()
        
        # Wait for connection with timeout
        start_time = time.time()
        while not app.connected and time.time() - start_time < timeout:
            time.sleep(0.1)
        
        if not app.connected:
            logger.error("Failed to connect to IBKR within timeout")
            app.disconnect()
            return None
        
        return app
    
    except Exception as e:
        logger.error(f"Error connecting to IBKR: {str(e)}")
        return None

def get_option_price(app, contract, timeout=5):
    """Get current price for an option contract"""
    if app.next_order_id is None:
        logger.error("Not connected to IBKR")
        return None
    
    # Reset event flag
    app.market_data_received.clear()
    
    # Request market data
    req_id = app.next_order_id
    app.next_order_id += 1
    
    try:
        logger.info(f"Requesting market data for {contract.symbol} {contract.right} {contract.strike}")
        app.reqMktData(req_id, contract, "", False, False, [])
        
        # Wait for market data with timeout
        got_data = app.market_data_received.wait(timeout)
        
        # Cancel market data request
        app.cancelMktData(req_id)
        
        if not got_data:
            logger.warning(f"Timeout waiting for market data")
            return None
        
        # Return premium data
        if req_id in app.option_data:
            bid = app.option_data[req_id].get(1)
            ask = app.option_data[req_id].get(2)
            return {'bid': bid, 'ask': ask}
        
        return None
    
    except Exception as e:
        logger.error(f"Error getting option price: {str(e)}")
        return None

def place_option_order(app, contract, action, quantity, price, timeout=10):
    """Place an option order"""
    if app.next_order_id is None:
        logger.error("Not connected to IBKR")
        return None
    
    # Reset event flag
    app.order_placed_event.clear()
    
    # Create the order
    order = Order()
    order.action = action
    order.totalQuantity = quantity
    order.orderType = "LMT"
    order.lmtPrice = price
    order.tif = "DAY"
    
    # Get the next valid order ID
    order_id = app.next_order_id
    app.next_order_id += 1
    
    try:
        # Place the order
        logger.info(f"Placing {action} order for {quantity} {contract.symbol} {contract.right} {contract.strike} at ${price}")
        app.placeOrder(order_id, contract, order)
        
        # Wait for order confirmation with timeout
        placed = app.order_placed_event.wait(timeout)
        
        if not placed:
            logger.error("Timeout waiting for order status")
            return None
        
        # Return order info
        if order_id in app.order_status:
            return {
                'order_id': order_id,
                'status': app.order_status[order_id]['status'],
                'price': app.order_status[order_id]['avgFillPrice']
            }
        
        return None
    
    except Exception as e:
        logger.error(f"Error placing order: {str(e)}")
        return None

def get_triggered_strategies(db_path):
    """Get strategies with 'triggered' status from the database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Check if necessary columns exist
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [column[1] for column in cursor.fetchall()]
        
        if 'strategy_status' not in columns:
            logger.error("Database does not have strategy_status column")
            conn.close()
            return []
        
        # Query for triggered strategies
        cursor.execute("""
            SELECT id, ticker, strategy_type, strike_price, options_expiry_date, estimated_premium, price_when_triggered
            FROM option_strategies 
            WHERE strategy_status = 'triggered'
        """)
        
        strategies = []
        for row in cursor.fetchall():
            strategy = {
                'id': row[0],
                'ticker': row[1],
                'strategy_type': row[2],
                'strike_price': row[3],
                'options_expiry_date': row[4],
                'estimated_premium': row[5],
                'price_when_triggered': row[6]
            }
            strategies.append(strategy)
        
        conn.close()
        logger.info(f"Found {len(strategies)} triggered strategies")
        return strategies
    
    except Exception as e:
        logger.error(f"Error querying database: {str(e)}")
        return []

def update_strategy_status(db_path, strategy_id, new_status, premium_received=None):
    """Update strategy status and premium received in the database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Check if premium_received column exists
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [column[1] for column in cursor.fetchall()]
        
        if 'premium_received' not in columns:
            cursor.execute("ALTER TABLE option_strategies ADD COLUMN premium_received REAL")
        
        # Update the strategy status
        if premium_received is not None:
            cursor.execute(
                "UPDATE option_strategies SET strategy_status = ?, premium_received = ? WHERE id = ?",
                (new_status, premium_received, strategy_id)
            )
        else:
            cursor.execute(
                "UPDATE option_strategies SET strategy_status = ? WHERE id = ?",
                (new_status, strategy_id)
            )
        
        conn.commit()
        conn.close()
        logger.info(f"Updated strategy {strategy_id} status to '{new_status}'")
        return True
    
    except Exception as e:
        logger.error(f"Error updating database: {str(e)}")
        return False

def process_triggered_strategies(db_path, ibkr_host='127.0.0.1', ibkr_port=7497):
    """Process all triggered strategies and place orders"""
    # Connect to IBKR
    app = connect_to_ibkr(host=ibkr_host, port=ibkr_port)
    if not app:
        logger.error("Failed to connect to IBKR")
        return
    
    try:
        # Get triggered strategies
        strategies = get_triggered_strategies(db_path)
        if not strategies:
            logger.info("No triggered strategies to process")
            return
        
        # Process each strategy
        for strategy in strategies:
            strategy_id = strategy['id']
            ticker = strategy['ticker']
            strategy_type = strategy['strategy_type']
            
            # Handle different formats of strike price
            try:
                if isinstance(strategy['strike_price'], str):
                    strike_price = float(strategy['strike_price'].replace('$', '').replace(',', '').strip())
                else:
                    strike_price = float(strategy['strike_price'])
            except (ValueError, TypeError):
                logger.warning(f"Invalid strike price format: {strategy['strike_price']}")
                continue
                
            # Handle different formats of expiry date
            try:
                expiry_date = strategy['options_expiry_date']
                if expiry_date and '-' in expiry_date:
                    expiry_date = expiry_date.replace('-', '')  # Format as YYYYMMDD
            except (ValueError, TypeError):
                logger.warning(f"Invalid expiry date format: {strategy['options_expiry_date']}")
                continue
                
            # Handle different formats of estimated premium
            try:
                if isinstance(strategy['estimated_premium'], str) and strategy['estimated_premium']:
                    estimated_premium = float(strategy['estimated_premium'].replace('$', '').replace(',', '').strip())
                elif strategy['estimated_premium'] is not None:
                    estimated_premium = float(strategy['estimated_premium'])
                else:
                    estimated_premium = 0.5  # Default value if not specified
            except (ValueError, TypeError):
                logger.warning(f"Invalid estimated premium format: {strategy['estimated_premium']}")
                estimated_premium = 0.5  # Default fallback
            
            logger.info(f"Processing {strategy_type} for {ticker}, strike ${strike_price}, expiry {expiry_date}")
            
            # Determine option parameters based on strategy
            if strategy_type == 'Bear Call':
                option_type = "C"  # Call
                action = "SELL"
            elif strategy_type == 'Bull Put':
                option_type = "P"  # Put
                action = "SELL"
            else:
                logger.warning(f"Unknown strategy type: {strategy_type}")
                continue
            
            # Create option contract
            contract = create_option_contract(ticker, expiry_date, strike_price, option_type)
            
            # Get current option price
            price_data = get_option_price(app, contract)
            if not price_data or price_data['bid'] is None:
                logger.warning(f"Could not get option price for {ticker}")
                continue
            
            # For sell orders, we use the bid price
            premium = price_data['bid']
            
            # Compare with estimated premium
            if premium < estimated_premium:
                logger.warning(f"Current premium (${premium}) is less than estimated premium (${estimated_premium})")
                update_strategy_status(db_path, strategy_id, 'premium too low')
                continue
            
            # Place the order
            quantity = 1  # Default to 1 contract
            order_result = place_option_order(app, contract, action, quantity, premium)
            
            if order_result and order_result['status'] in ['Submitted', 'Filled']:
                logger.info(f"Order placed successfully at ${premium}")
                update_strategy_status(db_path, strategy_id, 'order placed', premium)
            else:
                logger.warning("Order placement failed")
                update_strategy_status(db_path, strategy_id, 'order failed')
    
    except Exception as e:
        logger.error(f"Error processing strategies: {str(e)}")
    
    finally:
        # Disconnect from IBKR
        if app:
            app.disconnect()
            logger.info("Disconnected from IBKR")

# Configuration variables - customize these
db_path = './database/option_strategies.db'  # Path to your database
ibkr_host = '127.0.0.1'  # IBKR host
ibkr_port = 7497  # 7497 for paper trading, 7496 for live trading

# Run the process
process_triggered_strategies(db_path, ibkr_host, ibkr_port)

2025-03-12 21:30:53,593 - INFO - sent startApi
2025-03-12 21:30:53,594 - INFO - REQUEST startApi {}
2025-03-12 21:30:53,594 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x001\x00\x00'
2025-03-12 21:30:53,595 - INFO - ANSWER connectAck {}
2025-03-12 21:30:53,598 - INFO - Connected to IBKR, next order ID: 1
2025-03-12 21:30:53,626 - INFO - ANSWER managedAccounts {'accountsList': 'DU9233079'}
2025-03-12 21:30:53,630 - ERROR - Error 1741815054477: 2104
2025-03-12 21:30:53,634 - ERROR - Error 1741815054478: 2104
2025-03-12 21:30:53,637 - ERROR - Error 1741815054478: 2104
2025-03-12 21:30:53,640 - ERROR - Error 1741815054481: 2104
2025-03-12 21:30:53,645 - ERROR - Error 1741815054489: 2104
2025-03-12 21:30:53,647 - ERROR - Error 1741815054489: 2104
2025-03-12 21:30:53,649 - ERROR - Error 1741815054489: 2104
2025-03-12 21:30:53,651 - ERROR - Error 1741815054490: 2104
2025-03-12 21:30:53,654 - ERROR - Error 1741815054492: 2106
2025-03-12 21:30:53,657 - ERROR - Error 1741815054494: 2106
2

In [15]:
import sqlite3
import logging
import threading
import time
from datetime import datetime

# IBKR API imports
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract, ComboLeg
from ibapi.order import Order

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()


class OptionOrderApp(EWrapper, EClient):
    """Simple IBKR options order application"""

    def __init__(self):
        EClient.__init__(self, self)
        self.next_order_id = None
        self.order_status = {}
        self.option_data = {}
        self.order_placed_event = threading.Event()
        self.contract_details_received = threading.Event()
        self.market_data_received = threading.Event()
        self.connected = False

    def nextValidId(self, orderId):
        """Callback for next valid order ID"""
        self.next_order_id = orderId
        self.connected = True
        logger.info(f"Connected to IBKR, next order ID: {orderId}")

    def error(self, reqId, errorCode, errorString, *args):
        """Error handling callback"""
        if errorCode not in [2104, 2106]:  # Filter out common notifications
            logger.error(f"Error {errorCode}: {errorString}")

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, *args):
        """Order status update callback"""
        self.order_status[orderId] = {
            'status': status,
            'filled': filled,
            'remaining': remaining,
            'avgFillPrice': avgFillPrice,
            'time': datetime.now(),
        }
        logger.info(f"Order {orderId} status: {status}, filled: {filled}, price: {avgFillPrice}")

        if status in ["Submitted", "Filled", "Cancelled"]:
            self.order_placed_event.set()

    def tickPrice(self, reqId, tickType, price, attrib):
        """Price data callback"""
        if reqId not in self.option_data:
            self.option_data[reqId] = {}

        # 1 = bid, 2 = ask, 4 = last, 9 = close
        self.option_data[reqId][tickType] = price

        # Signal that we have market data if both bid and ask are received
        if 1 in self.option_data[reqId] and 2 in self.option_data[reqId]:
            self.market_data_received.set()


def create_option_contract(ticker, expiry_date, strike, option_type):
    """
    Create a standard option contract object.
    This is used for market data requests on the sold leg.
    """
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "OPT"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = expiry_date  # formatted as YYYYMMDD
    contract.strike = float(strike)
    contract.right = option_type
    contract.multiplier = "100"
    return contract


def create_vertical_spread_contract(ticker, expiry_date, strike_buy, strike_sell, option_type):
    """
    Create a vertical spread combo contract.
    The sold leg (strike_sell) is the one we use for pricing,
    while the bought leg is at strike_buy.
    """
    combo_contract = Contract()
    combo_contract.symbol = ticker
    combo_contract.secType = "BAG"
    combo_contract.currency = "USD"
    combo_contract.exchange = "SMART"
    combo_contract.lastTradeDateOrContractMonth = expiry_date

    # NOTE: In a production environment you must resolve the proper conId for each leg.
    # Here we assume a helper function 'resolve_conid' exists.
    sold_leg = ComboLeg()
    sold_leg.conId = resolve_conid(ticker, expiry_date, strike_sell, option_type)
    sold_leg.ratio = 1
    sold_leg.action = "SELL"
    sold_leg.exchange = "SMART"

    bought_leg = ComboLeg()
    bought_leg.conId = resolve_conid(ticker, expiry_date, strike_buy, option_type)
    bought_leg.ratio = 1
    bought_leg.action = "BUY"
    bought_leg.exchange = "SMART"

    combo_contract.comboLegs = [sold_leg, bought_leg]
    return combo_contract


def resolve_conid(ticker, expiry_date, strike, option_type):
    """
    Placeholder function to resolve the contract ID (conId) for an option.
    In practice, you would query IBKR for contract details.
    """
    # For this example, we simply return 0.
    return 0


def connect_to_ibkr(host='127.0.0.1', port=7497, client_id=1, timeout=10):
    """Connect to IBKR TWS/Gateway"""
    app = OptionOrderApp()
    try:
        app.connect(host, port, client_id)
        # Start API thread
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()

        # Wait for connection with timeout
        start_time = time.time()
        while not app.connected and time.time() - start_time < timeout:
            time.sleep(0.1)

        if not app.connected:
            logger.error("Failed to connect to IBKR within timeout")
            app.disconnect()
            return None

        return app

    except Exception as e:
        logger.error(f"Error connecting to IBKR: {str(e)}")
        return None


def get_option_price(app, contract, timeout=5):
    """
    Get current price for an option contract.
    For vertical spreads, we request market data on the sold leg.
    """
    if app.next_order_id is None:
        logger.error("Not connected to IBKR")
        return None

    # Reset event flag
    app.market_data_received.clear()

    req_id = app.next_order_id
    app.next_order_id += 1

    try:
        logger.info(f"Requesting market data for {contract.symbol} {contract.right} {contract.strike}")
        app.reqMktData(req_id, contract, "", False, False, [])
        # Wait for market data with timeout
        got_data = app.market_data_received.wait(timeout)
        # Cancel market data request
        app.cancelMktData(req_id)

        if not got_data:
            logger.warning("Timeout waiting for market data")
            return None

        if req_id in app.option_data:
            bid = app.option_data[req_id].get(1)
            ask = app.option_data[req_id].get(2)
            return {'bid': bid, 'ask': ask}
        return None

    except Exception as e:
        logger.error(f"Error getting option price: {str(e)}")
        return None


def place_option_order(app, contract, action, quantity, price, timeout=10):
    """Place an option order (combo order for vertical spread)"""
    if app.next_order_id is None:
        logger.error("Not connected to IBKR")
        return None

    app.order_placed_event.clear()

    order = Order()
    order.action = action
    order.totalQuantity = quantity
    order.orderType = "LMT"
    order.lmtPrice = price
    order.tif = "DAY"

    order_id = app.next_order_id
    app.next_order_id += 1

    try:
        logger.info(f"Placing {action} order for {quantity} combo spread order at ${price}")
        app.placeOrder(order_id, contract, order)
        placed = app.order_placed_event.wait(timeout)

        if not placed:
            logger.error("Timeout waiting for order status")
            return None

        if order_id in app.order_status:
            return {
                'order_id': order_id,
                'status': app.order_status[order_id]['status'],
                'price': app.order_status[order_id]['avgFillPrice']
            }
        return None

    except Exception as e:
        logger.error(f"Error placing order: {str(e)}")
        return None


def get_triggered_strategies(db_path):
    """
    Get strategies with 'triggered' status from the database.
    The query now retrieves strike_buy and strike_sell values.
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Check if necessary columns exist
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [column[1] for column in cursor.fetchall()]

        if 'strategy_status' not in columns:
            logger.error("Database does not have strategy_status column")
            conn.close()
            return []

        # Query for triggered strategies (now including strike_buy and strike_sell)
        cursor.execute("""
            SELECT id, ticker, strategy_type, strike_buy, strike_sell, options_expiry_date, estimated_premium, price_when_triggered
            FROM option_strategies 
            WHERE strategy_status = 'triggered'
        """)

        strategies = []
        for row in cursor.fetchall():
            strategy = {
                'id': row[0],
                'ticker': row[1],
                'strategy_type': row[2],
                'strike_buy': row[3],
                'strike_sell': row[4],
                'options_expiry_date': row[5],
                'estimated_premium': row[6],
                'price_when_triggered': row[7]
            }
            strategies.append(strategy)

        conn.close()
        logger.info(f"Found {len(strategies)} triggered strategies")
        return strategies

    except Exception as e:
        logger.error(f"Error querying database: {str(e)}")
        return []


def update_strategy_status(db_path, strategy_id, new_status, premium_received=None):
    """Update strategy status and premium received in the database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [column[1] for column in cursor.fetchall()]

        if 'premium_received' not in columns:
            cursor.execute("ALTER TABLE option_strategies ADD COLUMN premium_received REAL")

        if premium_received is not None:
            cursor.execute(
                "UPDATE option_strategies SET strategy_status = ?, premium_received = ? WHERE id = ?",
                (new_status, premium_received, strategy_id)
            )
        else:
            cursor.execute(
                "UPDATE option_strategies SET strategy_status = ? WHERE id = ?",
                (new_status, strategy_id)
            )

        conn.commit()
        conn.close()
        logger.info(f"Updated strategy {strategy_id} status to '{new_status}'")
        return True

    except Exception as e:
        logger.error(f"Error updating database: {str(e)}")
        return False


def process_triggered_strategies(db_path, ibkr_host='127.0.0.1', ibkr_port=7497):
    """Process all triggered strategies and place orders as vertical spreads"""
    app = connect_to_ibkr(host=ibkr_host, port=ibkr_port)
    if not app:
        logger.error("Failed to connect to IBKR")
        return

    try:
        strategies = get_triggered_strategies(db_path)
        if not strategies:
            logger.info("No triggered strategies to process")
            return

        for strategy in strategies:
            strategy_id = strategy['id']
            ticker = strategy['ticker']
            strategy_type = strategy['strategy_type']

            # Parse strike_buy and strike_sell
            try:
                if isinstance(strategy['strike_buy'], str):
                    strike_buy = float(strategy['strike_buy'].replace('$', '').replace(',', '').strip())
                else:
                    strike_buy = float(strategy['strike_buy'])
            except (ValueError, TypeError):
                logger.warning(f"Invalid strike_buy format: {strategy['strike_buy']}")
                continue

            try:
                if isinstance(strategy['strike_sell'], str):
                    strike_sell = float(strategy['strike_sell'].replace('$', '').replace(',', '').strip())
                else:
                    strike_sell = float(strategy['strike_sell'])
            except (ValueError, TypeError):
                logger.warning(f"Invalid strike_sell format: {strategy['strike_sell']}")
                continue

            # Format expiry date (expected as YYYYMMDD)
            try:
                expiry_date = strategy['options_expiry_date']
                if expiry_date and '-' in expiry_date:
                    expiry_date = expiry_date.replace('-', '')
            except (ValueError, TypeError):
                logger.warning(f"Invalid expiry date format: {strategy['options_expiry_date']}")
                continue

            # Parse estimated premium
            try:
                if isinstance(strategy['estimated_premium'], str) and strategy['estimated_premium']:
                    estimated_premium = float(strategy['estimated_premium'].replace('$', '').replace(',', '').strip())
                elif strategy['estimated_premium'] is not None:
                    estimated_premium = float(strategy['estimated_premium'])
                else:
                    estimated_premium = 0.5
            except (ValueError, TypeError):
                logger.warning(f"Invalid estimated premium format: {strategy['estimated_premium']}")
                estimated_premium = 0.5

            logger.info(f"Processing {strategy_type} for {ticker}, buy at ${strike_buy}, sell at ${strike_sell}, expiry {expiry_date}")

            # Determine option type based on strategy_type
            if strategy_type == 'Bear Call':
                option_type = "C"  # Calls
                action = "SELL"   # Sell the spread (credit spread)
            elif strategy_type == 'Bull Put':
                option_type = "P"  # Puts
                action = "SELL"   # Sell the spread (credit spread)
            else:
                logger.warning(f"Unknown strategy type: {strategy_type}")
                continue

            # Create vertical spread combo contract
            combo_contract = create_vertical_spread_contract(ticker, expiry_date, strike_buy, strike_sell, option_type)

            # For pricing, request market data on the sold leg
            sold_leg_contract = create_option_contract(ticker, expiry_date, strike_sell, option_type)
            price_data = get_option_price(app, sold_leg_contract)
            if not price_data or price_data['bid'] is None:
                logger.warning(f"Could not get market data for {ticker} sold leg")
                continue

            premium = price_data['bid']
            if premium < estimated_premium:
                logger.warning(f"Current premium (${premium}) is less than estimated premium (${estimated_premium})")
                update_strategy_status(db_path, strategy_id, 'premium too low')
                continue

            quantity = 1  # Default to one spread
            order_result = place_option_order(app, combo_contract, action, quantity, premium)
            if order_result and order_result['status'] in ['Submitted', 'Filled']:
                logger.info(f"Order placed successfully at ${premium}")
                update_strategy_status(db_path, strategy_id, 'order placed', premium)
            else:
                logger.warning("Order placement failed")
                update_strategy_status(db_path, strategy_id, 'order failed')

    except Exception as e:
        logger.error(f"Error processing strategies: {str(e)}")

    finally:
        if app:
            app.disconnect()
            logger.info("Disconnected from IBKR")


# Configuration variables
db_path = './database/option_strategies.db'  # Path to your database
ibkr_host = '127.0.0.1'  # IBKR host
ibkr_port = 7497       # 7497 for paper trading, 7496 for live trading

# Run the process
process_triggered_strategies(db_path, ibkr_host, ibkr_port)


2025-03-11 05:45:25,831 - INFO - sent startApi
2025-03-11 05:45:25,832 - INFO - REQUEST startApi {}
2025-03-11 05:45:25,833 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x001\x00\x00'
2025-03-11 05:45:25,834 - INFO - ANSWER connectAck {}
2025-03-11 05:45:25,838 - INFO - ANSWER managedAccounts {'accountsList': 'DU9233079'}
2025-03-11 05:45:25,861 - INFO - Connected to IBKR, next order ID: 1
2025-03-11 05:45:25,863 - ERROR - Error 1741671926932: 2104
2025-03-11 05:45:25,864 - ERROR - Error 1741671926932: 2104
2025-03-11 05:45:25,866 - ERROR - Error 1741671926933: 2104
2025-03-11 05:45:25,868 - ERROR - Error 1741671926933: 2104
2025-03-11 05:45:25,869 - ERROR - Error 1741671926933: 2104
2025-03-11 05:45:25,871 - ERROR - Error 1741671926934: 2104
2025-03-11 05:45:25,872 - ERROR - Error 1741671926934: 2104
2025-03-11 05:45:25,873 - ERROR - Error 1741671926934: 2104
2025-03-11 05:45:25,875 - ERROR - Error 1741671926934: 2104
2025-03-11 05:45:25,876 - ERROR - Error 1741671926935: 2106
2

In [2]:
import sqlite3
import logging
import threading
import time
from datetime import datetime

# IBKR API imports
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract, ComboLeg
from ibapi.order import Order

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()


class OptionOrderApp(EWrapper, EClient):
    """Simple IBKR options order application with contract details retrieval"""

    def __init__(self):
        EClient.__init__(self, self)
        self.next_order_id = None
        self.order_status = {}
        self.option_data = {}
        self.contract_details = {}  # Store contract details per request id
        self.order_placed_event = threading.Event()
        self.contract_details_event = threading.Event()
        self.market_data_received = threading.Event()
        self.connected = False

    def nextValidId(self, orderId):
        """Callback for next valid order ID"""
        self.next_order_id = orderId
        self.connected = True
        logger.info(f"Connected to IBKR, next order ID: {orderId}")

    def error(self, reqId, errorCode, errorString, *args):
        """Error handling callback"""
        # Filter out common notifications if needed.
        if errorCode not in [2104, 2106]:
            logger.error(f"Error {reqId}: {errorCode} {errorString}")

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, *args):
        """Order status update callback"""
        self.order_status[orderId] = {
            'status': status,
            'filled': filled,
            'remaining': remaining,
            'avgFillPrice': avgFillPrice,
            'time': datetime.now(),
        }
        logger.info(f"Order {orderId} status: {status}, filled: {filled}, price: {avgFillPrice}")

        if status in ["Submitted", "Filled", "Cancelled"]:
            self.order_placed_event.set()

    def tickPrice(self, reqId, tickType, price, attrib):
        """Price data callback"""
        if reqId not in self.option_data:
            self.option_data[reqId] = {}

        # 1 = bid, 2 = ask, 4 = last, 9 = close
        self.option_data[reqId][tickType] = price

        # Signal that we have market data if both bid and ask are received
        if 1 in self.option_data[reqId] and 2 in self.option_data[reqId]:
            self.market_data_received.set()

    def contractDetails(self, reqId, contractDetails):
        """Callback for contract details"""
        if reqId not in self.contract_details:
            self.contract_details[reqId] = []
        self.contract_details[reqId].append(contractDetails)
        # Use the 'contract' attribute rather than 'summary'
        logger.info(f"Received contract details for reqId {reqId}: {contractDetails.contract}")


    def contractDetailsEnd(self, reqId):
        """Callback indicating the end of contract details"""
        logger.info(f"Contract details request {reqId} completed")
        self.contract_details_event.set()


def create_option_contract(ticker, expiry_date, strike, option_type):
    """
    Create a standard option contract object.
    This is used for market data requests on the sold leg.
    """
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "OPT"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = expiry_date  # formatted as YYYYMMDD
    contract.strike = float(strike)
    contract.right = option_type
    contract.multiplier = "100"
    return contract


def create_vertical_spread_contract(ticker, expiry_date, strike_buy, strike_sell, option_type):
    """
    Create a vertical spread combo contract.
    The sold leg (strike_sell) is used for pricing,
    while the bought leg is at strike_buy.
    """
    combo_contract = Contract()
    combo_contract.symbol = ticker
    combo_contract.secType = "BAG"
    combo_contract.currency = "USD"
    combo_contract.exchange = "SMART"
    combo_contract.lastTradeDateOrContractMonth = expiry_date

    # NOTE: In a production environment you must resolve the proper conId for each leg.
    # Here we assume a helper function 'resolve_conid' exists.
    sold_leg = ComboLeg()
    sold_leg.conId = resolve_conid(ticker, expiry_date, strike_sell, option_type)
    sold_leg.ratio = 1
    sold_leg.action = "SELL"
    sold_leg.exchange = "SMART"

    bought_leg = ComboLeg()
    bought_leg.conId = resolve_conid(ticker, expiry_date, strike_buy, option_type)
    bought_leg.ratio = 1
    bought_leg.action = "BUY"
    bought_leg.exchange = "SMART"

    combo_contract.comboLegs = [sold_leg, bought_leg]
    return combo_contract


def resolve_conid(ticker, expiry_date, strike, option_type):
    """
    Placeholder function to resolve the contract ID (conId) for an option.
    In practice, you would query IBKR for contract details.
    """
    # For this example, we simply return 0.
    return 0


def connect_to_ibkr(host='127.0.0.1', port=7497, client_id=1, timeout=10):
    """Connect to IBKR TWS/Gateway"""
    app = OptionOrderApp()
    try:
        app.connect(host, port, client_id)
        # Start API thread
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()

        # Wait for connection with timeout
        start_time = time.time()
        while not app.connected and time.time() - start_time < timeout:
            time.sleep(0.1)

        if not app.connected:
            logger.error("Failed to connect to IBKR within timeout")
            app.disconnect()
            return None

        return app

    except Exception as e:
        logger.error(f"Error connecting to IBKR: {str(e)}")
        return None


def get_contract_details(app, contract, timeout=5):
    """
    Request contract details from IBKR to validate the contract.
    Returns a list of contract details if found, otherwise None.
    """
    req_id = app.next_order_id
    app.next_order_id += 1

    # Clear previous details and event flag
    if req_id in app.contract_details:
        del app.contract_details[req_id]
    app.contract_details_event.clear()

    try:
        logger.info(f"Requesting contract details for {contract.symbol} {contract.secType} {contract.lastTradeDateOrContractMonth} {contract.strike} {contract.right}")
        app.reqContractDetails(req_id, contract)

        # Wait for contract details to be received
        got_details = app.contract_details_event.wait(timeout)
        if not got_details or req_id not in app.contract_details:
            logger.error("Timeout waiting for contract details")
            return None

        return app.contract_details[req_id]

    except Exception as e:
        logger.error(f"Error requesting contract details: {str(e)}")
        return None


def get_option_price(app, contract, timeout=5):
    """
    Get current price for an option contract.
    Before requesting market data, query for contract details to ensure the contract is valid.
    """
    # Query IBKR for contract details first
    details = get_contract_details(app, contract, timeout)
    if not details:
        logger.error("No valid contract details found; aborting market data request")
        return None

    if app.next_order_id is None:
        logger.error("Not connected to IBKR")
        return None

    # Reset market data event flag
    app.market_data_received.clear()

    req_id = app.next_order_id
    app.next_order_id += 1

    try:
        logger.info(f"Requesting market data for {contract.symbol} {contract.right} {contract.strike}")
        app.reqMktData(req_id, contract, "", False, False, [])
        # Wait for market data with timeout
        got_data = app.market_data_received.wait(timeout)
        # Cancel market data request
        app.cancelMktData(req_id)

        if not got_data:
            logger.warning("Timeout waiting for market data")
            return None

        if req_id in app.option_data:
            bid = app.option_data[req_id].get(1)
            ask = app.option_data[req_id].get(2)
            return {'bid': bid, 'ask': ask}
        return None

    except Exception as e:
        logger.error(f"Error getting option price: {str(e)}")
        return None


def place_option_order(app, contract, action, quantity, price, timeout=10):
    """Place an option order (combo order for vertical spread)"""
    if app.next_order_id is None:
        logger.error("Not connected to IBKR")
        return None

    app.order_placed_event.clear()

    order = Order()
    order.action = action
    order.totalQuantity = quantity
    order.orderType = "LMT"
    order.lmtPrice = price
    order.tif = "DAY"

    order_id = app.next_order_id
    app.next_order_id += 1

    try:
        logger.info(f"Placing {action} order for {quantity} combo spread order at ${price}")
        app.placeOrder(order_id, contract, order)
        placed = app.order_placed_event.wait(timeout)

        if not placed:
            logger.error("Timeout waiting for order status")
            return None

        if order_id in app.order_status:
            return {
                'order_id': order_id,
                'status': app.order_status[order_id]['status'],
                'price': app.order_status[order_id]['avgFillPrice']
            }
        return None

    except Exception as e:
        logger.error(f"Error placing order: {str(e)}")
        return None


def get_triggered_strategies(db_path):
    """
    Get strategies with 'triggered' status from the database.
    The query now retrieves strike_buy and strike_sell values.
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Check if necessary columns exist
        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [column[1] for column in cursor.fetchall()]

        if 'strategy_status' not in columns:
            logger.error("Database does not have strategy_status column")
            conn.close()
            return []

        # Query for triggered strategies (now including strike_buy and strike_sell)
        cursor.execute("""
            SELECT id, ticker, strategy_type, strike_buy, strike_sell, options_expiry_date, estimated_premium, price_when_triggered
            FROM option_strategies 
            WHERE strategy_status = 'triggered'
        """)

        strategies = []
        for row in cursor.fetchall():
            strategy = {
                'id': row[0],
                'ticker': row[1],
                'strategy_type': row[2],
                'strike_buy': row[3],
                'strike_sell': row[4],
                'options_expiry_date': row[5],
                'estimated_premium': row[6],
                'price_when_triggered': row[7]
            }
            strategies.append(strategy)

        conn.close()
        logger.info(f"Found {len(strategies)} triggered strategies")
        return strategies

    except Exception as e:
        logger.error(f"Error querying database: {str(e)}")
        return []


def update_strategy_status(db_path, strategy_id, new_status, premium_received=None):
    """Update strategy status and premium received in the database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        cursor.execute("PRAGMA table_info(option_strategies)")
        columns = [column[1] for column in cursor.fetchall()]

        if 'premium_received' not in columns:
            cursor.execute("ALTER TABLE option_strategies ADD COLUMN premium_received REAL")

        if premium_received is not None:
            cursor.execute(
                "UPDATE option_strategies SET strategy_status = ?, premium_received = ? WHERE id = ?",
                (new_status, premium_received, strategy_id)
            )
        else:
            cursor.execute(
                "UPDATE option_strategies SET strategy_status = ? WHERE id = ?",
                (new_status, strategy_id)
            )

        conn.commit()
        conn.close()
        logger.info(f"Updated strategy {strategy_id} status to '{new_status}'")
        return True

    except Exception as e:
        logger.error(f"Error updating database: {str(e)}")
        return False


def process_triggered_strategies(db_path, ibkr_host='127.0.0.1', ibkr_port=7497):
    """Process all triggered strategies and place orders as vertical spreads"""
    app = connect_to_ibkr(host=ibkr_host, port=ibkr_port)
    if not app:
        logger.error("Failed to connect to IBKR")
        return

    try:
        strategies = get_triggered_strategies(db_path)
        if not strategies:
            logger.info("No triggered strategies to process")
            return

        for strategy in strategies:
            strategy_id = strategy['id']
            ticker = strategy['ticker']
            strategy_type = strategy['strategy_type']

            # Parse strike_buy and strike_sell
            try:
                if isinstance(strategy['strike_buy'], str):
                    strike_buy = float(strategy['strike_buy'].replace('$', '').replace(',', '').strip())
                else:
                    strike_buy = float(strategy['strike_buy'])
            except (ValueError, TypeError):
                logger.warning(f"Invalid strike_buy format: {strategy['strike_buy']}")
                continue

            try:
                if isinstance(strategy['strike_sell'], str):
                    strike_sell = float(strategy['strike_sell'].replace('$', '').replace(',', '').strip())
                else:
                    strike_sell = float(strategy['strike_sell'])
            except (ValueError, TypeError):
                logger.warning(f"Invalid strike_sell format: {strategy['strike_sell']}")
                continue

            # Format expiry date (expected as YYYYMMDD)
            try:
                expiry_date = strategy['options_expiry_date']
                if expiry_date and '-' in expiry_date:
                    expiry_date = expiry_date.replace('-', '')
            except (ValueError, TypeError):
                logger.warning(f"Invalid expiry date format: {strategy['options_expiry_date']}")
                continue

            # Parse estimated premium
            try:
                if isinstance(strategy['estimated_premium'], str) and strategy['estimated_premium']:
                    estimated_premium = float(strategy['estimated_premium'].replace('$', '').replace(',', '').strip())
                elif strategy['estimated_premium'] is not None:
                    estimated_premium = float(strategy['estimated_premium'])
                else:
                    estimated_premium = 0.5
            except (ValueError, TypeError):
                logger.warning(f"Invalid estimated premium format: {strategy['estimated_premium']}")
                estimated_premium = 0.5

            logger.info(f"Processing {strategy_type} for {ticker}, buy at ${strike_buy}, sell at ${strike_sell}, expiry {expiry_date}")

            # Determine option type based on strategy_type
            if strategy_type == 'Bear Call':
                option_type = "C"  # Calls
                action = "SELL"   # Sell the spread (credit spread)
            elif strategy_type == 'Bull Put':
                option_type = "P"  # Puts
                action = "SELL"   # Sell the spread (credit spread)
            else:
                logger.warning(f"Unknown strategy type: {strategy_type}")
                continue

            # Create vertical spread combo contract
            combo_contract = create_vertical_spread_contract(ticker, expiry_date, strike_buy, strike_sell, option_type)

            # For pricing, request market data on the sold leg.
            sold_leg_contract = create_option_contract(ticker, expiry_date, strike_sell, option_type)
            price_data = get_option_price(app, sold_leg_contract)
            if not price_data or price_data['bid'] is None:
                logger.warning(f"Could not get market data for {ticker} sold leg")
                continue

            premium = price_data['bid']
            if premium < estimated_premium:
                logger.warning(f"Current premium (${premium}) is less than estimated premium (${estimated_premium})")
                update_strategy_status(db_path, strategy_id, 'premium too low')
                continue

            quantity = 1  # Default to one spread
            order_result = place_option_order(app, combo_contract, action, quantity, premium)
            if order_result and order_result['status'] in ['Submitted', 'Filled']:
                logger.info(f"Order placed successfully at ${premium}")
                update_strategy_status(db_path, strategy_id, 'order placed', premium)
            else:
                logger.warning("Order placement failed")
                update_strategy_status(db_path, strategy_id, 'order failed')

    except Exception as e:
        logger.error(f"Error processing strategies: {str(e)}")

    finally:
        if app:
            app.disconnect()
            logger.info("Disconnected from IBKR")


# Configuration variables
db_path = './database/option_strategies.db'  # Path to your database
ibkr_host = '127.0.0.1'  # IBKR host
ibkr_port = 7497       # 7497 for paper trading, 7496 for live trading

# Run the process
process_triggered_strategies(db_path, ibkr_host, ibkr_port)


2025-03-12 21:31:04,995 - INFO - sent startApi
2025-03-12 21:31:04,996 - INFO - REQUEST startApi {}
2025-03-12 21:31:04,996 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x001\x00\x00'
2025-03-12 21:31:04,998 - INFO - ANSWER connectAck {}


2025-03-12 21:31:05,010 - INFO - Connected to IBKR, next order ID: 1
2025-03-12 21:31:05,031 - INFO - ANSWER managedAccounts {'accountsList': 'DU9233079'}
2025-03-12 21:31:05,032 - ERROR - Error -1: 1741815065880 2104
2025-03-12 21:31:05,033 - ERROR - Error -1: 1741815065881 2104
2025-03-12 21:31:05,034 - ERROR - Error -1: 1741815065881 2104
2025-03-12 21:31:05,035 - ERROR - Error -1: 1741815065882 2104
2025-03-12 21:31:05,035 - ERROR - Error -1: 1741815065882 2104
2025-03-12 21:31:05,037 - ERROR - Error -1: 1741815065882 2104
2025-03-12 21:31:05,038 - ERROR - Error -1: 1741815065882 2104
2025-03-12 21:31:05,039 - ERROR - Error -1: 1741815065882 2104
2025-03-12 21:31:05,040 - ERROR - Error -1: 1741815065882 2106
2025-03-12 21:31:05,042 - ERROR - Error -1: 1741815065883 2106
2025-03-12 21:31:05,043 - ERROR - Error -1: 1741815065883 2106
2025-03-12 21:31:05,045 - ERROR - Error -1: 1741815065883 2158
2025-03-12 21:31:05,111 - INFO - Found 0 triggered strategies
2025-03-12 21:31:05,112 - I